In [3]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Input, Dense,Dropout
from keras.models import Model
from keras import regularizers,initializers
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dropout

C:\Users\etri-sw-soc\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\etri-sw-soc\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\etri-sw-soc\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\etri-sw-soc\AppData\Roami

In [4]:
data = np.load('/Users/etri-sw-soc/Desktop/temp/data.npy',allow_pickle=True) #numpy배열로 저장된 배열 가져오기
label=np.load('/Users/etri-sw-soc/Desktop/temp/label.npy',allow_pickle=True)

In [5]:


data_nor=[]

for i in data: #min-max 전처리
    ii=[(float(i[j])) for j in range(1024)]
    
    temp2=[]
    minv=min(ii)
    maxv=max(ii)
    maxmin=maxv-minv
    for k in ii:
        temp2.append((k-minv)/maxmin)
    data_nor.append(temp2)   


data_nor=np.array(data_nor)

x_train, x_test, y_train, y_test = train_test_split(data_nor,label,test_size=0.3)
y_test_num=y_test[:50] #autoencoder 성능 확인용 test 배열(그래프 print용)
print( x_train.shape)
print( x_test.shape)

y_train=to_categorical(y_train,10) #라벨 형태 변환(십진수->010000형태)
y_test=to_categorical(y_test,10)
print(y_train.shape)

a=[0,0,0,0,0,0,0,0,0,0] #한 클래스 당 데이터 수 확인
for i in label:
    a[i]+=1
print(a)

(34419, 1024)
(14751, 1024)
(34419, 10)
[4800, 4800, 4800, 5568, 5250, 4752, 4800, 4800, 4800, 4800]


validation data 가져오기

In [6]:
#test data에서 train할때 사용할 validation용의 데이터 가져오기

x_test, x_val, y_test, y_val=train_test_split(x_test,y_test,test_size=0.4)
print(x_test.shape, x_val.shape)

(8850, 1024) (5901, 1024)


encoder, decoder생성

In [11]:
#autoencoder 생성
l2_regularizer =regularizers.l2(0.00001)    #tf.contrib.layers.l2_regularizer(scale=0.0001)

input_dim=1024
encoding_dim =512 #256의 경우 너무 많이 압축되어 autoencoder의 성능이 떨어짐.
dropout_rate=0.1

input_ = Input(shape=(input_dim,))
encoded = Dense(input_dim, activation=tf.nn.relu)(input_) #12_regularizer을 안쓰는 것이 성능이 더 좋은 경우도 O->제거
encoded=Dropout(dropout_rate)(encoded)
encoded=Dense(int(encoding_dim*1.5),activation=tf.nn.relu)(encoded)
encoded=Dropout(dropout_rate)(encoded)
decoded=Dense(encoding_dim,activation=tf.nn.relu)(encoded)
decoded=Dense(int(encoding_dim*1.5),activation=tf.nn.relu)(decoded)
decoded = Dense(input_dim, activation=tf.nn.relu)(decoded)

encoder=Model(input_,encoded)

autoencoder= Model(input_, decoded)

encoded_input=Input(shape=(encoding_dim,))
decoder_layer2=autoencoder.layers[-2]
decoder_layer1=autoencoder.layers[-1]
decoder=Model(encoded_input, decoder_layer1(decoder_layer2((encoded_input))))

#autoencoder.compile(optimizer='adam', loss='mean_squared_error',metrics=['mse'])
autoencoder.compile(optimizer='adam', loss='mean_squared_error',metrics=['mse','accuracy']) #optimizer=Adam(lr=0.002)
auto_train=autoencoder.fit(x_train, x_train, epochs=700, batch_size=350,shuffle=True,validation_data=(x_val,x_val))


Train on 34419 samples, validate on 5901 samples
Epoch 1/700
34419/34419 [==============================] - 10s 288us/step - loss: 0.0601 - mean_squared_error: 0.0601 - acc: 6.9729e-04 - val_loss: 0.0462 - val_mean_squared_error: 0.0462 - val_acc: 0.0024
Epoch 2/700
34419/34419 [==============================] - 9s 271us/step - loss: 0.0380 - mean_squared_error: 0.0380 - acc: 0.0012 - val_loss: 0.0313 - val_mean_squared_error: 0.0313 - val_acc: 0.0019
Epoch 3/700
34419/34419 [==============================] - 11s 305us/step - loss: 0.0283 - mean_squared_error: 0.0283 - acc: 0.0012 - val_loss: 0.0286 - val_mean_squared_error: 0.0286 - val_acc: 0.0017
Epoch 4/700
34419/34419 [==============================] - 10s 303us/step - loss: 0.0259 - mean_squared_error: 0.0259 - acc: 0.0015 - val_loss: 0.0265 - val_mean_squared_error: 0.0265 - val_acc: 0.0012
Epoch 5/700
34419/34419 [==============================] - 10s 296us/step - loss: 0.0245 - mean_squared_error: 0.0245 - acc: 0.0022 - val_lo

KeyboardInterrupt: 

encoder, decoder plot

In [ ]:
import matplotlib.pyplot as plt
n = 10

def plotting(startpoint,n): #autoencoder성능 확인용 함수 (기존 데이터와 재생성 데이터 plot)
    print('실제 라벨 : ',y_test_num[startpoint:startpoint+n]) #정답 라벨 프린트
    plt.figure(figsize=(20, 4))
    
    for i in range( n):
        # original data
        ax = plt.subplot(2, n, i + 1) 
        plt.plot(x_test[startpoint+i])
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        plt.plot(decoded_result[startpoint+i])
        #plt.plot(x_test_noise[startpoint+i])
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()


In [ ]:

encoded_result=encoder.predict(x_test[:30]) #autoencoder 성능 확인용 print
decoded_result=decoder.predict(encoded_result)

plotting(0,10) #test 데이터에 autoencoder적용 결과 plot(test데이터의 0~10번째 데이터)
plotting(10,10) #test데이터의 10~20번째 데이터 출력

layer추가

In [ ]:
#Layer추가, Fine tuning
dropout_rate_=0.2 #얼마나 dropout할지

model=Sequential()
count=0
for layer in encoder.layers: #model에 encoder부분 추가하기
    model.add(layer)
        
for layer in model.layers: #encoder부분은 더이상 학습X
    layer.trainable=False
     
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(dropout_rate_))
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(dropout_rate_))
model.add(Dense(10,activation='softmax'))

model.summary()

In [48]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])#Adam(lr=0.002)
hist=model.fit(x_train, y_train, batch_size=350, epochs=300, verbose=2, validation_data=(x_val, y_val))

Train on 34419 samples, validate on 5901 samples
Epoch 1/300
 - 8s - loss: 1.8046 - acc: 0.3346 - val_loss: 1.5550 - val_acc: 0.4276
Epoch 2/300
 - 7s - loss: 1.5072 - acc: 0.4467 - val_loss: 1.4022 - val_acc: 0.4875
Epoch 3/300
 - 7s - loss: 1.3581 - acc: 0.4978 - val_loss: 1.2421 - val_acc: 0.5477
Epoch 4/300
 - 7s - loss: 1.2436 - acc: 0.5446 - val_loss: 1.1752 - val_acc: 0.5723
Epoch 5/300
 - 7s - loss: 1.1461 - acc: 0.5792 - val_loss: 1.1017 - val_acc: 0.5994
Epoch 6/300
 - 7s - loss: 1.0588 - acc: 0.6113 - val_loss: 1.0379 - val_acc: 0.6219
Epoch 7/300
 - 7s - loss: 1.0004 - acc: 0.6393 - val_loss: 0.9691 - val_acc: 0.6487
Epoch 8/300
 - 7s - loss: 0.9378 - acc: 0.6593 - val_loss: 0.9424 - val_acc: 0.6587
Epoch 9/300
 - 7s - loss: 0.8768 - acc: 0.6853 - val_loss: 0.9274 - val_acc: 0.6682
Epoch 10/300
 - 7s - loss: 0.8355 - acc: 0.6997 - val_loss: 0.8667 - val_acc: 0.6904
Epoch 11/300
 - 7s - loss: 0.7978 - acc: 0.7114 - val_loss: 0.8994 - val_acc: 0.6723
Epoch 12/300
 - 7s - loss

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

#test loss, test accuracy print
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print("test loss : ",test_loss," test acc : ", test_acc) #정확도 대략 80%

#train loss plot
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()

#train accuracy plot
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model acc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['acc', 'val_acc'], loc='upper left')
plt.show()